# importar as bibliotecas

In [ ]:
import spacy
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# executar as funções

In [ ]:
def searchSimilar(phrase, Comments) -> list:
	filterdData = []
	
	nlp = spacy.load("pt_core_news_sm")
	corpus_old = phrase.split() + (list(Comments['content'].values)) #concatena a phrase na base dados
	
	for i in corpus_old:
		filterdData.append(clearPhrase(i, nlp, ['PROPN', 'NOUN', 'VERB'])) #eu faço uma limpeza nos comentários, deixando apenas Pronomes pessoais, adjetivos, substantivos e verbos 
		
	processedData = TfidfVectorizer().fit_transform(filterdData) #transformo a base de dados em vetores de números TF-IDF
	
	[lenght, M] = processedData.shape #pego o tamanho da base de dados    
	similaritiesList = []    
	for i in range(1, lenght):
		s = cosine_similarity(processedData[0],processedData[i]) #o comentário processedData[0] é a frase digitada pelo usuário, daí preciso computar a distancia dessa frase com todos os comentários da base de dados
		if(s != 0): #se a similaridade é zero, nem adiciono o comentário na lista, pois significa que ele é 0 similar
			similaritiesList.append([s, corpus_old[i]])
		
	return similaritiesList   #retorna uma lista com o comentário e a distancia dele da frase que o usuário digitou
	
def clearPhrase(doc, nlp, pos_tag):
	if(type(doc) != spacy.tokens.doc.Doc):
		doc = nlp(doc)
	return ' '.join([str(t.lemma_) for t in doc if t.pos_ in pos_tag]) 

def addHours(minDate:str,maxDate:str)-> (str,str):
	minDate = minDate + "00:00:00"
	maxDate = maxDate + "23:59:59"

	return minDate,maxDate
def getComments(originPath:str,minDate:str = None,maxDate:str = None,versions = None) -> pandas.DataFrame:
	
	commentsFile = pandas.read_csv(originPath)

	if minDate != None and maxDate != None:
		minDate,maxDate = addHours(minDate,maxDate)

	for i in range(0, len(commentsFile)):
		if commentsFile['date'][i] < minDate or commentsFile['date'][i] > maxDate or commentsFile['version'][i] not in versions:
			#print("droped",commentsFile['version'][i])
			commentsFile = commentsFile.drop([i])
	return commentsFile

def sortbyThumbs(file:pandas.DataFrame) -> pandas.DataFrame:
	file = file.sort_values(by=['thumbs'])
	return file


# coletar os comentários

In [ ]:
database = getComments("./General_Data.csv","2021-09-01","2021-10-01","3.0.0")

# buscar por comentários similares

In [ ]:
similarities_list = searchSimilar('recuperar senha', database)

In [ ]:
print(sorted(similarities_list))

[[array([[0.13738488]]), 'Boa tarde me chamo Valdelice, não estou conseguindo entrar nas secretárias da prefeitura e nem acessar órgãos do governo, eu não fiz a foto e todo vez d erro de senha . Realmente não consigo resolver nada . E pior preciso da entrada na licença ambiental de um cliente e não consigo, preciso também providenciar o alvará de funcionamento também não consigo. Já fiz de tudo para recuperar e não consigo oi seja um aplicativo que era pra facilitar infelizmente está prejudicando'], [array([[0.15780862]]), 'Estou a dias tentando recuperar uma senha que foi cadastrada em meu nome e não tenho acesso ao e-mail para recuperação,não tenho acesso ao número para receber o sms com o código . Se tento criar uma conta o CPF já existe . Será que por favo vocês poderiam me ajudar . Obrigado e aguardo retorno.'], [array([[0.20724942]]), 'Não lembro minha senha, a única forma de recuperar e em um número de telefone que não tenho mais acesso! As outras opções não dá também! Pois não 